# Exam Reference — 1MS041 (copy‑paste ready)
**What this is:** compact “toolbox + mini‑textbook” for the exam: definitions, formulas (plain + LaTeX), and *ready-to-copy* Python templates.  
**What this is NOT:** solutions to any specific past exam.

**How to use in the exam**
- Jump to the relevant chapter.
- Copy the code block, then only edit: data arrays, distribution functions, matrix values.
- Keep comments: they help you earn partial credit.


## Quick index
1. **Sampling + Monte Carlo integration + Hoeffding CI**
2. **Logistic regression + calibration + 0–1 loss + CI**
3. **Markov chains** (transition matrix, irreducible, periodic, stationary, reversible)


# 1) Sampling + Monte Carlo integration (MC)

## 1.1 Definitions (summary like a textbook)

### Random variable generation
Goal: generate i.i.d. samples \(X_1,\dots,X_n\) from a target distribution with CDF \(F\) or density \(f\).

### Inverse transform sampling
If \(F^{-1}\) is available:
- draw \(U\sim \mathrm{Unif}(0,1)\)
- set \(X = F^{-1}(U)\)
Then \(X\) has CDF \(F\).

### Rejection sampling (accept–reject)
Use when you can evaluate \(f(x)\) (possibly up to a constant) but cannot sample directly.

Pick:
- proposal density \(g(x)\) that you can sample from,
- constant \(M\) such that \(f(x) \le M g(x)\) for all \(x\) in support.

Algorithm:
1. Sample \(Y\sim g\).
2. Sample \(U\sim \mathrm{Unif}(0,1)\).
3. Accept \(Y\) if \(U \le f(Y)/(M g(Y))\); else reject and repeat.

Acceptance rate is about \(1/M\). Choosing a good \(g\) is key.

### Monte Carlo integration
To approximate \(I=\mathbb{E}[h(X)]\) with samples \(X_i\sim f\):
\[
\hat I = \frac{1}{n}\sum_{i=1}^n h(X_i).
\]
Importance sampling with \(Y_i\sim g\):
\[
I = \int h(x) f(x)\,dx = \mathbb{E}_g\!\left[h(Y)\frac{f(Y)}{g(Y)}\right],
\quad
\hat I = \frac{1}{n}\sum_{i=1}^n h(Y_i)\frac{f(Y_i)}{g(Y_i)}.
\]

### Hoeffding inequality (bounded variables)
If \(Z_i\in[a,b]\) i.i.d. and \(\hat\mu=\frac{1}{n}\sum Z_i\):
\[
\Pr(|\hat\mu-\mu|\ge \varepsilon)\le 2\exp\!\left(\frac{-2n\varepsilon^2}{(b-a)^2}\right).
\]
CI:
\[
\hat\mu \pm (b-a)\sqrt{\frac{\ln(2/\delta)}{2n}}.
\]

---

## 1.2 Copyable formulas (plain text)
- Inverse sampling: U ~ Unif(0,1), X = F^{-1}(U).
- Reject sampling: accept Y~g with prob f(Y)/(M*g(Y)).
- MC: I ≈ (1/n) Σ h(X_i).
- Importance: I = int h(x) f(x) dx ≈ (1/n) Σ h(Y_i) * f(Y_i)/g(Y_i).
- Hoeffding CI: mean ± (b-a)*sqrt(log(2/delta)/(2n)).


In [ ]:
# 1.3 Imports for this chapter
import numpy as np


## 1.4 Code templates (copy‑paste)

### A) Rejection sampler (vectorized)
Fill in:
- g_sampler(size)
- f_unnorm(x)
- g_pdf(x)
- M


In [ ]:
def rejection_sample(n_samples, g_sampler, f_unnorm, g_pdf, M, batch=10000, rng=None):
    # Generic accept-reject sampler returning exactly n_samples accepted draws.
    #
    # Parameters:
    # - n_samples: number of accepted samples to output
    # - g_sampler(size): draws from proposal distribution g
    # - f_unnorm(x): target density (can be unnormalized), must be >= 0
    # - g_pdf(x): proposal density g(x), must be >= 0
    # - M: constant with f_unnorm(x) <= M * g_pdf(x) over support
    # - batch: batch size for vectorized proposals (speed)
    # - rng: np.random.Generator for reproducibility

    if rng is None:
        rng = np.random.default_rng()

    accepted_chunks = []
    n_acc = 0

    while n_acc < n_samples:
        # 1) Propose candidates
        y = g_sampler(batch)

        # 2) Uniforms for accept step
        u = rng.random(batch)

        # 3) Compute acceptance probability f(y)/(M*g(y))
        gy = g_pdf(y)
        fy = f_unnorm(y)

        # Avoid division by zero (e.g., if g_pdf(y)=0 at boundaries)
        accept_prob = fy / (M * gy + 1e-300)

        # 4) Accept
        mask = u <= accept_prob
        if np.any(mask):
            accepted = y[mask]
            accepted_chunks.append(accepted)
            n_acc += accepted.size

    # Concatenate and trim to exact size
    return np.concatenate(accepted_chunks)[:n_samples]


### B) Inverse CDF sampling
You provide Finv(u) = F^{-1}(u).


In [ ]:
def inverse_cdf_sample(n_samples, Finv, rng=None):
    # Inverse transform sampling: X = F^{-1}(U), U ~ Uniform(0,1).
    if rng is None:
        rng = np.random.default_rng()
    u = rng.random(n_samples)
    return Finv(u)


### C) Monte Carlo integration


In [ ]:
def mc_integral_direct(samples, h):
    # Direct MC: I = E[h(X)] ≈ mean(h(samples)).
    vals = h(samples)
    return float(np.mean(vals))

def mc_integral_importance(samples_from_g, h, f_pdf, g_pdf):
    # Importance sampling: I = E_g[h(Y)*f(Y)/g(Y)].
    y = samples_from_g
    w = f_pdf(y) / (g_pdf(y) + 1e-300)
    vals = h(y) * w
    return float(np.mean(vals))


### D) Hoeffding CI
LaTeX:
\[
\hat\mu \pm (b-a)\sqrt{\frac{\ln(2/\delta)}{2n}}
\]


In [ ]:
def hoeffding_ci(mean_estimate, n, a, b, delta=0.05):
    # Two-sided Hoeffding CI for bounded variables in [a,b].
    radius = (b - a) * np.sqrt(np.log(2.0 / delta) / (2.0 * n))
    return float(mean_estimate - radius), float(mean_estimate + radius)


# 2) Logistic regression + calibration + 0–1 loss CI

## 2.1 Definitions (summary)

### Logistic regression
\[
P(Y=1|X)=\sigma(\beta_0+\beta^\top X),\quad \sigma(t)=\frac{1}{1+e^{-t}}.
\]

### Negative log-likelihood (cross-entropy)
\[
\mathcal{L} = -\sum_{i=1}^n \left[y_i\log p_i + (1-y_i)\log(1-p_i)\right],\quad p_i=\sigma(\beta_0+\beta^\top x_i).
\]

### Calibration
Learn a mapping from raw predicted probabilities \(\hat p\) to calibrated probabilities \(\tilde p\) using a separate calibration set.

### 0–1 loss
\[
\hat L = \frac{1}{n}\sum_{i=1}^n \mathbf{1}\{\hat y_i\ne y_i\},\quad \hat y_i=\mathbf{1}\{\tilde p_i\ge 0.5\}.
\]
Hoeffding (with a=0,b=1) gives:
\[
\hat L \pm \sqrt{\frac{\ln(2/\delta)}{2n}}.
\]

---

## 2.2 Copyable formulas (plain text)
- sigma(t)=1/(1+exp(-t))
- p_i = sigma(beta0 + beta^T x_i)
- NLL = -Σ [ y_i log p_i + (1-y_i) log(1-p_i) ]
- predict: yhat = 1 if p>=0.5 else 0
- 0-1 loss = mean(yhat != y)
- CI: loss ± sqrt(log(2/delta)/(2n))


In [ ]:
# 2.3 Imports for this chapter
import numpy as np


In [ ]:
def sigmoid(t):
    # Numerically stable sigmoid: clip to avoid exp overflow.
    t = np.clip(t, -50, 50)
    return 1.0 / (1.0 + np.exp(-t))


## 2.4 Code templates

### A) Logistic NLL + training with scipy.optimize.minimize


In [ ]:
def logistic_nll(theta, X, y, l2=0.0):
    # Negative log-likelihood for logistic regression (optionally with L2 penalty).
    # theta[0] = intercept beta0, theta[1:] = beta vector.

    beta0 = theta[0]
    beta = theta[1:]

    # Linear predictor
    z = beta0 + X @ beta

    # Probabilities
    p = sigmoid(z)

    # Avoid log(0)
    eps = 1e-12
    p = np.clip(p, eps, 1 - eps)

    # NLL (cross-entropy)
    nll = -np.sum(y * np.log(p) + (1 - y) * np.log(1 - p))

    # L2 penalty on beta only (not intercept)
    if l2 > 0:
        nll += l2 * np.sum(beta ** 2)

    return float(nll)


In [ ]:
def fit_logistic(X, y, method="CG", l2=0.0):
    # Fit logistic regression by minimizing NLL.
    # Returns (theta_hat, scipy_result).
    from scipy import optimize

    n, d = X.shape
    theta0 = np.zeros(d + 1)

    obj = lambda th: logistic_nll(th, X, y, l2=l2)
    res = optimize.minimize(obj, theta0, method=method)
    return res.x, res


In [ ]:
def predict_proba_logistic(theta, X):
    # Predict probabilities P(Y=1|X) given theta.
    beta0 = theta[0]
    beta = theta[1:]
    return sigmoid(beta0 + X @ beta)


### B) Calibration (DecisionTreeRegressor)


In [ ]:
def fit_tree_calibrator(p_cal, y_cal, max_depth=3, min_samples_leaf=50, random_state=0):
    # Fit decision-tree regressor mapping raw probabilities -> calibrated probabilities.
    from sklearn.tree import DecisionTreeRegressor

    X_feat = p_cal.reshape(-1, 1)

    calibrator = DecisionTreeRegressor(
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=random_state
    )
    calibrator.fit(X_feat, y_cal)
    return calibrator

def apply_calibrator(calibrator, p_raw):
    # Apply calibrator and clip to [0,1].
    p_cal = calibrator.predict(p_raw.reshape(-1, 1))
    return np.clip(p_cal, 0.0, 1.0)


### C) 0–1 loss + Hoeffding CI


In [ ]:
def decision_from_proba(p, threshold=0.5):
    # Predict class labels from probabilities under equal costs.
    return (p >= threshold).astype(int)

def zero_one_loss(y_true, y_pred):
    # 0-1 loss = mean of misclassification indicator.
    return float(np.mean(y_true != y_pred))

def hoeffding_ci_01(loss_estimate, n, delta=0.05):
    # Hoeffding CI for 0-1 loss (bounded in [0,1]).
    radius = np.sqrt(np.log(2.0 / delta) / (2.0 * n))
    return float(loss_estimate - radius), float(loss_estimate + radius)


# 3) Markov chains (finite state)

## 3.1 Definitions (summary)

### Transition matrix
$$

P_{ij} = \Pr(X_{t+1}=j \mid X_t=i).

$$
Rows sum to 1.

### Irreducible
$$

\forall i,j,\ \exists n\ge1 \text{ such that } (P^n)_{ij}>0.

$$
Graph view: strongly connected directed graph of positive edges.

### Period
$$

d(i)=\gcd\{n\ge1:(P^n)_{ii}>0\}.

$$
Aperiodic means period 1. In irreducible chains, all states share the same period.

### Stationary distribution
$$

\pi=\pi P,\quad \sum_i\pi_i=1.

$$
Finite + irreducible implies existence and uniqueness.

### Reversibility (detailed balance)
$$

\pi_i P_{ij} = \pi_j P_{ji}\ \ \forall i,j.

$$

---

## 3.2 Copyable formulas (plain text)
- Stationary: solve pi = pi*P with sum(pi)=1.
- Period: d(i)=gcd{ n>=1 : (P^n)[i,i] > 0 }.
- Reversible: pi_i P_ij = pi_j P_ji for all i,j.


In [ ]:
# 3.3 Imports for this chapter
import numpy as np


## 3.4 Code templates (Markov chains)


In [ ]:
def is_irreducible(P, tol=1e-15):
    # Check irreducibility using reachability on the directed graph.
    P = np.asarray(P, dtype=float)
    n = P.shape[0]
    adj = (P > tol)

    for start in range(n):
        seen = {start}
        stack = [start]
        while stack:
            i = stack.pop()
            for j in range(n):
                if adj[i, j] and j not in seen:
                    seen.add(j)
                    stack.append(j)
        if len(seen) != n:
            return False
    return True


In [ ]:
def periods(P, max_power=200, tol=1e-15):
    # Compute state periods d(i) by collecting return times up to max_power.
    P = np.asarray(P, dtype=float)
    n = P.shape[0]

    returns = [[] for _ in range(n)]
    Pk = np.eye(n)

    for k in range(1, max_power + 1):
        Pk = Pk @ P
        diag = np.diag(Pk)
        for i in range(n):
            if diag[i] > tol:
                returns[i].append(k)

    def gcd_list(lst):
        from math import gcd
        g = lst[0]
        for x in lst[1:]:
            g = gcd(g, x)
        return g

    out = np.zeros(n, dtype=object)
    for i in range(n):
        out[i] = None if len(returns[i]) == 0 else gcd_list(returns[i])
    return out

def is_aperiodic(P):
    # Aperiodic if all states have period 1 (assuming computation found returns).
    per = periods(P)
    if any(p is None for p in per):
        return False
    return all(int(p) == 1 for p in per)


In [ ]:
def stationary_distribution(P):
    # Solve pi = pi P with sum(pi)=1 using a linear system.
    P = np.asarray(P, dtype=float)
    n = P.shape[0]

    A = P.T - np.eye(n)
    A[-1, :] = 1.0
    b = np.zeros(n)
    b[-1] = 1.0

    pi = np.linalg.solve(A, b)
    pi = np.clip(pi, 0.0, 1.0)
    pi = pi / pi.sum()
    return pi


In [ ]:
def is_reversible(P, pi=None, tol=1e-8):
    # Check detailed balance pi_i P_ij == pi_j P_ji.
    P = np.asarray(P, dtype=float)
    if pi is None:
        pi = stationary_distribution(P)

    lhs = pi.reshape(-1, 1) * P
    rhs = (pi.reshape(1, -1) * P.T)
    return bool(np.allclose(lhs, rhs, atol=tol, rtol=0))


## 3.5 Markov chain answer checklist (copy into exam text)
1. Write $P$ and verify row sums.
2. Irreducible? argue reachability / strongly connected.
3. Periods: compute gcd of return times; self-loop implies period 1.
4. Stationary: solve $\pi=\pi P$, $\sum\pi=1$.
5. Reversible: check detailed balance.


## 3.6 Detailed Markov chain explanation with worked example (copy-friendly)

### What is a Markov chain?
A **(discrete-time) Markov chain** is a stochastic process $(X_t)_{t\ge 0}$ taking values in a finite set of states
$S=\{1,2,\dots,n\}$ such that the **Markov property** holds:

Plain text:
- P(X_{t+1}=j | X_t=i, X_{t-1},...,X_0) = P(X_{t+1}=j | X_t=i) = P_ij

LaTeX:
$$

\Pr(X_{t+1}=j \mid X_t=i, X_{t-1},\dots,X_0)=\Pr(X_{t+1}=j \mid X_t=i)=P_{ij}.

$$

Interpretation:
- “The future depends only on the present, not on the full history.”

---

### Q1. Transition matrix
Definition:
- The **transition matrix** $P\in\mathbb{R}^{n\times n}$ has entries $P_{ij}=\Pr(X_{t+1}=j\mid X_t=i)$.

Plain text:
- Each row i lists probabilities of moving out of state i.
- Rows sum to 1 and entries are ≥ 0.

LaTeX:
$$

P_{ij}=\Pr(X_{t+1}=j\mid X_t=i),\qquad \sum_{j=1}^n P_{ij}=1,\; P_{ij}\ge 0.

$$

How to build from a diagram:
1. Fix an ordering of states (A,B,C,...) → indices (0,1,2,...).
2. For each state, read outgoing arrows and probabilities.
3. Fill row i accordingly (use 0 where no arrow exists).

---

### Q2. Irreducible?
Definitions:
- State j is **reachable** from i if there exists $k\ge 1$ with $(P^k)_{ij}>0$.
- i and j **communicate** if i reaches j and j reaches i.
- The chain is **irreducible** if every pair of states communicates (one communicating class).

Plain text quick check:
- In the state graph, can you get from every node to every other node following directed edges?
- If yes → irreducible. If it splits into separate “closed” parts → reducible.

LaTeX:
$$

i\to j \iff \exists k\ge 1:\ (P^k)_{ij}>0,\qquad
\text{irreducible} \iff \forall i,j:\ i\to j.

$$

---

### Q3. Aperiodic? Period of each state
Return times:
- Look at the set $T_i=\{k\ge 1 : (P^k)_{ii}>0\}$, i.e., times when you can return to i.

Period of state i:
Plain text:
- d(i) = gcd of all return times to i.

LaTeX:
$$

d(i)=\gcd\{k\ge 1:\ (P^k)_{ii}>0\}.

$$

Aperiodic:
- State i is **aperiodic** if $d(i)=1$.
- In a **finite irreducible** chain, all states have the same period, so it’s enough to find one state with period 1.

Fast exam rules:
- If any state has a self-loop $P_{ii}>0$, then that state has period 1.
- In an irreducible chain, a single self-loop implies the whole chain is aperiodic.

---

### Q4. Stationary distribution (exists? what is it?)
Definition:
- A probability vector $\pi$ is **stationary** if $\pi=\pi P$.

Plain text:
- If X_0 ~ π, then X_t ~ π for all t.

LaTeX:
$$

\pi=\pi P,\qquad \sum_{i=1}^n \pi_i=1,\ \pi_i\ge 0.

$$

Existence/uniqueness (finite case):
- If the chain is **irreducible** and finite → a stationary distribution exists and is unique.
- If also **aperiodic**, then $P^t$ converges to π (long-run convergence).

How to compute:
1. Solve the linear system $\pi=\pi P$.
2. Add the normalization constraint $\sum_i \pi_i=1$.
3. (Practical) Replace one equation in $\pi=\pi P$ with the normalization to avoid singularity.

---

### Q5. Reversible?
Definition (detailed balance):
- A chain is **reversible** w.r.t. π if for all i,j:
  π_i P_ij = π_j P_ji.

LaTeX:
$$

\pi_i P_{ij}=\pi_j P_{ji}\quad \forall i,j.

$$

How to check fast:
1. Find π (stationary distribution).
2. Check detailed balance only on edges (pairs where $P_{ij}>0$ or $P_{ji}>0$).
3. One violation → not reversible.

---

## Worked example (3 states)
Suppose states are (A,B,C) and transitions are:
- From A: A→A with 0.2, A→B with 0.8
- From B: B→A with 0.5, B→C with 0.5
- From C: C→B with 1.0

### Transition matrix
$$

P=
\begin{pmatrix}
0.2 & 0.8 & 0\\
0.5 & 0 & 0.5\\
0 & 1.0 & 0
\end{pmatrix}.

$$

### Irreducible?
- A reaches C via A→B→C.
- C reaches A via C→B→A.
- All states reach each other → **irreducible**.

### Periods / aperiodic?
- A has self-loop (P_AA=0.2>0) ⇒ d(A)=1.
- Irreducible ⇒ all states have period 1.
So the chain is **aperiodic**.

### Stationary distribution
Solve $\pi=\pi P$ with $\pi_A+\pi_B+\pi_C=1$. The solution is:
$$

\pi=\left(\frac{5}{17},\frac{8}{17},\frac{4}{17}\right).

$$

### Reversibility
Check detailed balance on edges:
- A↔B: $\pi_A P_{AB}=(5/17)\cdot 0.8=4/17$, $\pi_B P_{BA}=(8/17)\cdot 0.5=4/17$.
- B↔C: $\pi_B P_{BC}=(8/17)\cdot 0.5=4/17$, $\pi_C P_{CB}=(4/17)\cdot 1=4/17$.
All checks pass ⇒ **reversible**.

---

### Mini answer template (paste into exam)
Plain text:
1) Transition matrix: write P with stated order, verify rows sum to 1.
2) Irreducible: argue reachability/one communicating class (or show a closed class).
3) Periods: compute gcd of return times; self-loop ⇒ period 1; if irreducible then all same.
4) Stationary: solve pi = pi P + sum(pi)=1; exists uniquely if finite + irreducible.
5) Reversible: check detailed balance pi_i P_ij = pi_j P_ji on edges.


# Appendix: sanity checks (optional)
Run during preparation to verify helper functions.


In [ ]:
P_example = np.array([[0.9, 0.1],
                      [0.4, 0.6]])

print("Irreducible:", is_irreducible(P_example))
print("Periods:", periods(P_example))
pi_ex = stationary_distribution(P_example)
print("Stationary pi:", pi_ex, "sum=", pi_ex.sum())
print("Reversible:", is_reversible(P_example, pi_ex))


## LaTeX-safe exam template (copy/paste)

Use **$...$** for inline math and **$$...$$** for display math.  
Avoid relying on `\(...\)` / `\[...\]` because some viewers don’t render them.

### Definitions (copy-friendly)
**Plain text:**  
- State space: S = {1,2,...,n}  
- Transition matrix entry: P_ij = P(X_{t+1}=j | X_t=i)

**LaTeX (inline):**  
- State space: $S=\{1,2,\dots,n\}$  
- Entry: $P_{ij}=\Pr(X_{t+1}=j\mid X_t=i)$

### One key equation (display)
$$
\pi = \pi P, \qquad \sum_{i=1}^n \pi_i = 1, \qquad \pi_i\ge 0.
$$

### Short reasoning template
- **Step 1 (matrix):** “Order states as (A,B,...) and fill each row with outgoing probabilities; rows sum to 1.”  
- **Step 2 (irreducible):** “Check reachability/strong connectivity in the directed graph.”  
- **Step 3 (period):** “Compute $d(i)=\gcd\{n\ge 1:(P^n)_{ii}>0\}$; self-loop implies $d(i)=1$.”  
- **Step 4 (stationary):** “Solve $\pi=\pi P$ plus normalization.”  
- **Step 5 (reversible):** “Check detailed balance $\pi_iP_{ij}=\pi_jP_{ji}$ on each edge.”

### Common pitfalls
- Writing math outside $...$ / $$...$$ (won’t render).
- Putting math inside code blocks ```...``` (won’t render).
- Forgetting the normalization equation for $\pi$.
